Tagline Generator with Feedback
    Input: Company name + product. 
    Generator creates a tagline. 
    Checker tests if it is catchy. 
    If not catchy → refine again.,

In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph , START , END
from typing import TypedDict

In [2]:
model = ChatOpenAI(model='gpt-5-nano')

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI

class WorkflowState(dict):
    tagline: str
    feedback: str
    approved: bool

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

def generate_tagline(state: WorkflowState):
    company = state.get("company")
    product = state.get("product")
    feedback = state.get("feedback", "")
    prompt = f"""
    Company: {company}
    Product: {product}
    {f"Previous feedback: {feedback}" if feedback else ""}
    Generate a short, catchy tagline.
    """
    tagline = llm.predict(prompt)
    state["tagline"] = tagline.strip()
    return state

def check_tagline(state: WorkflowState):
    tagline = state["tagline"]
    check_prompt = f"""
    Tagline: "{tagline}"
    Is this tagline short, memorable, and exciting? 
    Answer only with 'yes' or 'no' and give 1 short feedback sentence if 'no'.
    """
    result = llm.predict(check_prompt).lower()
    if "yes" in result:
        state["approved"] = True
    else:
        state["approved"] = False
        state["feedback"] = result
    return state

workflow = StateGraph(WorkflowState)

# Add nodes
workflow.add_node("generate", generate_tagline)
workflow.add_node("check", check_tagline)

# Add edges
workflow.add_edge("generate", "check")
workflow.add_conditional_edges(
    "check",
    lambda s: END if s.get("approved") else "generate"
)

workflow.set_entry_point("generate")

if __name__ == "__main__":
    initial_state = WorkflowState(company="EcoLife", product="reusable water bottles")
    app = workflow.compile()
    final_state = app.invoke(initial_state)
    print(f"✅ Final tagline: {final_state['tagline']}")


C:\Users\sujal\AppData\Local\Temp\ipykernel_25044\3817203380.py:24: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  tagline = llm.predict(prompt)


✅ Final tagline: "Shaping Tomorrow, Today!"
